In [ ]:
!pip install sentence-transformers
!pip install duckduckgo-search
!pip install nltk
!pip install selenium
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from nltk import word_tokenize, ngrams
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
import numpy as np
from duckduckgo_search import DDGS
import nltk
import random
from bs4 import BeautifulSoup
import itertools
import re
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def get_google_results(query,region):
    ddgs = DDGS()


    ddgs_text_gen = ddgs.text(query, region=region, safesearch='on', timelimit='y')

    titles=[]
    body=[]
    links=[]
    for r in ddgs_text_gen:
        titles.append(r['title'])
        body.append(r['body'])
        links.append(r['href'])
    raw_text = titles + body

    def remove_stop_words(text):
        stop_words = stopwords.words('english')
        return ' '.join([word for word in text.split() if word not in stop_words])

    titles = [remove_stop_words(title) for title in titles]
    all_titles="".join(titles)

    body = [remove_stop_words(text) for text in body]
    all_body="".join(body)

    all_text = all_titles + all_body
    all_text=" ".join(list(set([i.lower() for i in all_text.split()])))



    query_tokens = word_tokenize(query.lower())
    keywords=word_tokenize(all_text)
    keywords = [token for token in keywords if token.isalpha()]
    keywords = [token for token in keywords if token not in stopwords.words('english')]
    keywords = [token for token in keywords if len(token)>2]
    important_keywords = [token for token in keywords if token not in query_tokens]

    important_keywords = list(set(important_keywords))

    return important_keywords,links,raw_text

In [ ]:
def cosine_similarity(query,keywords):
    from sentence_transformers import SentenceTransformer
    import numpy as np
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence = query

    sentence_embedding = model.encode([sentence])[0]
    keyword_embeddings = model.encode(keywords)
    # Calculate cosine similarity
    similarities = np.dot(keyword_embeddings, sentence_embedding) / (np.linalg.norm(keyword_embeddings, axis=1) * np.linalg.norm(sentence_embedding))
    dic={}
    for keyword, similarity in zip(keywords, similarities):
        # print(f"Keyword: {keyword}\nSimilarity: {similarity}")
        dic[keyword]=similarity
    return dic

def get_top_keywords(query,keywords):
    dic = cosine_similarity(query,keywords)
    print(dic)
    top_keywords = sorted(dic, key=dic.get, reverse=True)[:100]
    return top_keywords

In [ ]:
!apt-get update
!apt-get install -y firefox xvfb
!pip install pyvirtualdisplay selenium

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
#import undetected_chromedriver as uc

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 3s (89.0 kB/s)
Reading package lists... Done


In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.ahpeyNHuuB/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.6FORvYv21J/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.1gF79Cv0Pf/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 http://deb.debian.org/debian-security buster/updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state info

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def firefox(url):
  try:
      response = requests.get(url)
      content = response.content.decode("utf-8")
      if response.status_code != 200:
            raise Exception("Request failed with status code: " + str(response.status_code))
      soup = BeautifulSoup(content, "html.parser")
      meta_title=''
      if soup.head.title is not None:
          meta_title = soup.head.title.get_text()
      description=''
      # # Extract the meta description
      meta_description = soup.find("meta", attrs={"name": "description"})
      # #print("Meta Description:", meta_description)
      if meta_description is not None:
          description=meta_description.get("content")
      # # Extract the body text excluding navbar, footer, and links
      excluded_tags = ["nav", "footer"]
      body_text = ""
      for tag in excluded_tags:
          for element in soup.find_all(tag):
              element.decompose()  # Remove the excluded tags from the soup

      body_text = soup.get_text(separator=" ")
      #print("Body Text (excluding navbar, footer, and links):", body_text)
      text = soup.get_text()
      text=re.sub(r"\W+", " ", text)
      # Extract the headings separately
      heading1 = [heading.get_text() for heading in soup.find_all(["h1"])]
      heading2 = [heading.get_text() for heading in soup.find_all(["h2",])]
      heading3 = [heading.get_text() for heading in soup.find_all(["h3"])]
      heading4 = [heading.get_text() for heading in soup.find_all(["h4"])]
      heading5 = [heading.get_text() for heading in soup.find_all(["h5"])]
      heading6 = [heading.get_text() for heading in soup.find_all(["h6"])]
      paragraphs = len([heading.get_text() for heading in soup.find_all(["p"])])
      images = len([heading.get_text() for heading in soup.find_all(["img"])])
      words=len(body_text.split())
      return {'text':meta_title+description+body_text,'heading1':heading1,'heading2':heading2,'heading3':heading3,'heading4':heading4,'heading5':heading5,'heading6':heading6,
              'paras':paragraphs,'imgs':images,'words':words}
  except Exception as e:
      print("An exception occurred:", str(e))


      # Set Firefox options
      # options = Options()
      # options.add_argument("--headless=new")
      # options.add_argument("--no-sandbox")
      # options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")


      # Set the path to the geckodriver executable (Firefox WebDriver)
      #webdriver_path = '/usr/local/bin/geckodriver'  # Assuming geckodriver is already installed in this path
      #service = Service(webdriver_path)

      # Instantiate the Firefox WebDriver
      #driver = webdriver.Firefox(options=firefox_options, executable_path=webdriver_path)
      #driver = webdriver.Firefox(service=service, options=firefox_options)
      # driver_service = Service('/usr/local/bin/chromedriver.exe')
      # driver = webdriver.Chrome(service=driver_service, options=options)

      # Load the website
      options = webdriver.ChromeOptions()
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      # options.add_argument('--disable-dev-shm-usage')
      options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36')

      # open it, go to a website, and get results
      driver = webdriver.Chrome(options=options)
      driver.get(url)
      wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed

      # Wait until the page is loaded and Cloudflare checking is complete
      wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, "div[class*='Ray ID']")))

      # options = uc.ChromeOptions()
      # options.headless = True
      # driver = uc.Chrome(use_subprocess=True, options=options)
      driver.get(url)
      page_source = driver.page_source
      soup = BeautifulSoup(page_source, "html.parser")

      # Extract the meta title
      meta_title=''
      if soup.head.title is not None:
         meta_title = soup.head.title.get_text()
      # print("Meta Title:", meta_title)
      description=''
      # # Extract the meta description
      meta_description = soup.find("meta", attrs={"name": "description"})
      # #print("Meta Description:", meta_description)
      if meta_description is not None:
          description=meta_description.get("content")
      # # Extract the body text excluding navbar, footer, and links
      excluded_tags = ["nav", "footer"]
      body_text = ""
      for tag in excluded_tags:
          for element in soup.find_all(tag):
              element.decompose()  # Remove the excluded tags from the soup

      body_text = soup.get_text(separator=" ")
      #print("Body Text (excluding navbar, footer, and links):", body_text)
      text = soup.get_text()
      text=re.sub(r"\W+", " ", text)
      # Extract the headings separately
      #headings = [heading.get_text() for heading in soup.find_all(["h1", "h2", "h3"])]
      #print("Headings:", headings)
      driver.close()

      heading1 = [heading.get_text() for heading in soup.find_all(["h1"])]
      heading2 = [heading.get_text() for heading in soup.find_all(["h2",])]
      heading3 = [heading.get_text() for heading in soup.find_all(["h3"])]
      heading4 = [heading.get_text() for heading in soup.find_all(["h4"])]
      heading5 = [heading.get_text() for heading in soup.find_all(["h5"])]
      heading6 = [heading.get_text() for heading in soup.find_all(["h6"])]
      paragraphs = len([heading.get_text() for heading in soup.find_all(["p"])])
      images = len([heading.get_text() for heading in soup.find_all(["img"])])
      words=len(body_text.split())
      return {'text':meta_title+description+body_text,'heading1':heading1,'heading2':heading2,'heading3':heading3,'heading4':heading4,'heading5':heading5,'heading6':heading6,
              'paras':paragraphs,'imgs':images,'words':words}

In [ ]:
print(firefox('https://www.weareteachers.com/educational-youtube-channels/'))

In [ ]:
import string

def extract_context(corpus, target_word,uncleanedwebsites=''):
    words = uncleanedwebsites.lower().split()  # Split the corpus into individual words
    keywords = []
    stemmer = PorterStemmer()
    unwebsites=uncleanedwebsites.lower()
    # main term/topic
    keywords.append({target_word:unwebsites.count(target_word)})
    stopwords_list = stopwords.words('english')  # Load English stopwords

    for i in range(len(words)):
        if stemmer.stem(words[i]) == stemmer.stem(target_word):
            context=[]
            # Extract the previous two and next two words if available
            prev_words = words[max(0, i - 2) : i]
            next_words = words[i + 1 : i + 3]
            context.extend(prev_words)
            context.append(words[i])
            context.extend(next_words)
            #merge two same words in array
            context = [word for word in context if word not in string.punctuation]
            #remove word if it's a stopword
            filtered_words = []

            for word in context:
              if word.lower() in stopwords_list or (len(word) == 1 and not word.isalnum()):
                      continue
              filtered_words.append(word)
            merged_words = []
            for word in filtered_words:
                if len(merged_words) > 0 and merged_words[-1] == word:
                     continue  # Skip consecutive duplicate word
                merged_words.append(word)
            #print(context)
            # Define your array
            my_array = merged_words
            combinationArray=[]
            # Iterate through all possible combinations
            for r in range(2, len(my_array) + 1):
                 combinations = list(itertools.combinations(my_array, r))
                 for combination in combinations:
                    if len(combination) >= 1:
                         combinationArray.append({' '.join(combination):unwebsites.count(' '.join(combination))})
                         #print(' '.join(combination),' :',unwebsites.count(' '.join(combination)))
                         reversed_combination = tuple(reversed(combination))
                         if reversed_combination != combination:
                              combinationArray.append({' '.join(reversed_combination):unwebsites.count(' '.join(reversed_combination))})
                             # print(' '.join(reversed_combination),':',unwebsites.count(' '.join(reversed_combination)))
                 sorted_data = sorted(combinationArray, key=lambda x: list(x.values())[0], reverse=True)
                 keywords.append(sorted_data[0])


    return keywords

In [ ]:
def nlp_terms(updated_words,text,uncleanedwebsites):
  keywords=[]
  for word in updated_words:
    #extract keywords
    corpus = '. '.join(text)
    target_word = word
    results = extract_context(corpus, target_word,'. '.join(uncleanedwebsites))
    sorted_data = sorted(results, key=lambda x: list(x.values())[0], reverse=True)
    cleaned_data = []
    for dictionary in sorted_data:
        cleaned_dictionary = {}
        for key, value in dictionary.items():
            # cleaned_key = remove_punctuation(key)
            cleaned_key = re.sub(r'[^\w\s]', '', key)
            cleaned_key = " ".join([word for word in cleaned_key.split() if word not in set(stopwords.words('english'))])
            cleaned_dictionary[cleaned_key] = cleaned_dictionary.get(cleaned_key, 0) + value
        cleaned_data.append(cleaned_dictionary)
    merged_data = []
    seen_keys = set()
    for d in cleaned_data:
        key = list(d.keys())[0]
        if key not in seen_keys:
            seen_keys.add(key)
            merged_data.append(d)
    filtered_data = [d for d in merged_data if any(word in key.lower() or word in str(value).lower() for key, value in d.items())]
    filtered_data = [d for d in filtered_data if any('. '.join(uncleanedwebsites).lower().count(key.lower()) > 0 for key, value in d.items())]
    sorted_data = sorted([item for item in filtered_data if list(item.values())[0] > 0], key=lambda x: list(x.values())[0], reverse=True)
    for w in sorted_data:
      keywords.append(list(w)[0])
    #keywords[word]=sorted_data[:30]
    # for i in range(len(keywords)):
    # #print(find_paragraphs_with_word(text, term))
    #     keywords[i] = [keywords[i]]
    #     keywords[i].append({'example':find_paragraphs_with_word('. '.join(uncleanedwebsites), keywords[i][0])})
  return keywords


In [ ]:
import re

def find_paragraphs_with_word(text, search_word):
    paragraphs_with_word = []
    text=text.lower()
    search_word=search_word.lower()
    paragraphs = re.split(r'\.\s*', text)  # Splitting by periods followed by optional whitespace to identify paragraphs

    search_pattern = re.compile(r'\b' + re.escape(search_word) + r'\b')  # Create a regular expression pattern for the search word

    for paragraph in paragraphs:
        if search_pattern.search(paragraph):
            paragraph_with_bold = search_pattern.sub(f'<b>{search_word}</b>', paragraph)
            paragraphs_with_word.append(paragraph_with_bold.strip())

    return paragraphs_with_word

def is_heading(paragraphs, search_word):
    paragraphs_with_word = []

    for paragraph in paragraphs:
        if search_word.lower() in paragraph.lower():
            return True


    return False


In [ ]:
!ngrok config add-authtoken 2hKmpjA51728jzC1Tg8Go0uXqJ7_3fDjcz77FtHj7pY444oW1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from googlesearch import search
from nltk.tokenize import sent_tokenize

app = Flask(__name__)

@app.route('/generalInfo', methods=['POST'])
def my_api():
    data = request.get_json()

    output=[]
    query = data['query']
    region=data['region']
    keywords,links ,raw_text= get_google_results(query,region)
    print(raw_text)
   # print(detect_language(raw_text))
    unique_keywords = list(set(get_top_keywords(query,keywords)))
    # query_tokens = word_tokenize(remove_stopwords(query.lower()))
    query_tokens = word_tokenize(" ".join([word for word in query.split() if word not in set(stopwords.words('english'))]))
   # unique_keywords = [keyword for keyword in unique_keywords if keyword not in query_tokens]
    output.append({'urls':links[:10],'nlp':query_tokens+unique_keywords})
    return jsonify(output)

@app.route('/website', methods=['POST'])
def websiteKeywords():
      data = request.get_json()
      text=firefox(data['url'])
      terms=nlp_terms(data['unique_keywords'],[text['text']],[text['text']])
      print(terms)
      for i in range(len(terms)):
        #print(find_paragraphs_with_word(text, term))
        terms[i] = [terms[i]]
        terms[i].append({'example':find_paragraphs_with_word(text['text'], terms[i][0]),'isheading':is_heading(text['heading1']+text['heading2']+text['heading3'],terms[i][0])})
      terms.insert(0,[data['query'],{'example':find_paragraphs_with_word(text['text'], data['query']),'isheading':is_heading(text['heading1']+text['heading2']+text['heading3'],data['query'])}])
      terms.append({'heading1':text['heading1'],'heading2':text['heading2'],'heading3':text['heading3'],'heading4':text['heading4'],'heading5':text['heading5'],'heading6':text['heading6'],'para':text['paras']
                    ,'imgs':text['imgs'],'words':text['words']})
      return jsonify(terms)

if __name__ == '__main__':
    ngrok_tunnel = ngrok.connect(5000)
    public_url = ngrok_tunnel.public_url
    print('Public URL:', public_url)
    app.run()

Public URL: https://d856-34-90-105-164.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


['What Is an Engineer? (Types, Salaries and Responsibilities)', 'What Do Engineers Do? Specialties, Roles and Duties - Indeed', 'Engineering - Wikipedia', 'What Is a Systems Engineer (and How Do I Become One)? - Coursera', 'How To Become an Engineer | Indeed.com', 'What Does a Civil Engineer Do? (+ How to Become One)', 'Careers in Engineering - NASA', 'What Are the Different Types of Engineering? | BestColleges', 'How To Become an Engineer | GCU Blog - Grand Canyon University', 'How to Become a Software Engineer | ComputerScience.org', 'What Does an Engineer Do? (Plus 12 Types of Engineers)', '11 Different Engineering Career Paths | Indeed.com', 'How Do I Become a Project Engineer? A Career Overview', 'What You Need to Know About Becoming an Engineering Major', 'What Are The Different Types Of Engineering Degrees? 11 Tracks To ...', 'Different Types of Engineering: Salary and Job Outlook | SNHU', 'Top Careers in Engineering (With Salaries, Job Duties and ... - Indeed', 'Salary: Enginee

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

['wireless', 'wireless c', 'wireless communication', 'wireless networks', 'modern wireless', 'wireless devices', 'wireless coverage', 'wireless communications', 'communication', 'wireless communication', 'radio communications', 'communication handling', 'communication effectively', 'frequency', 'frequency band', 'frequency bands', 'frequency spectrum', 'radio frequency', 'available frequency', 'technologies', 'cellular technologies', 'lpwan technologies', 'predictive technologies', 'new technologies', 'spectrum', 'frequency spectrum', 'equipment', 'necessary equipment', 'antennas', 'communicate', 'effectively communicate', 'communicate seamlessly', 'signal', 'signal strength', 'microcell signals', 'signal weakens', 'tech', 'voice', 'voice routing', 'phone', 'transmit', 'enabling', 'devices', 'mobile devices', 'wireless devices', 'specifications', '3gpp specifications', 'technical specifications', 'mechanism', 'critical mechanism', 'cellular', 'cellular network', 'cellular networks', 'c

INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 13:15:49] "POST /website HTTP/1.1" 200 -


['wireless', 'wireless communication', 'wireless network', 'wireless system', 'wireless service', 'used wireless', 'wireless systems', 'wireless telegraphy', 'wireless may', 'wireless lan', 'wireless channel', 'wireless networks', 'wireless bluetooth', 'best wireless', 'communication', 'wireless communication', 'communication system', 'communication systems', 'wired communication', 'satellite communication', 'infrared communication', 'way communication', 'telephone communication', 'mobile communication', 'communication services', 'simplex communication', 'radio communication', 'communication failure', 'communication infrastructure', 'uart communication', 'frequency', 'radio frequency', 'transmitted', 'transmitted message', 'easily transmitted', 'technologies', 'technologies use', 'spectrum', 'equipment', 'audio equipment', 'optical', 'optical fiber', 'antennas', 'wave', 'electromagnetic wave', 'electromagnetic waves', 'radio wave', 'radio waves', 'infrared waves', 'signal', 'electrical

INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 13:17:13] "POST /website HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 13:17:48] "POST /website HTTP/1.1" 200 -


['wireless', 'wireless communication', 'wireless mechanism', 'need wireless', 'wireless devices', 'communication', 'wireless communication', 'communication system', 'communication systems', 'mobile communication', 'communication protocol', 'communication channel', 'satellite communication', 'communication technology', 'wired communication', 'communication technologies', 'communication using', 'communication broadcast', 'frequency', 'frequency range', 'high frequency', 'frequency modulation', 'zigbee frequency', 'transmitted', 'information transmitted', 'data transmitted', 'transmitted even', 'technologies', 'modulation technologies', 'communication technologies', 'spectrum', 'smartphones', 'optical', 'antennas', 'wave', 'radio wave', 'radio waves', 'electromagnetic waves', 'waves enter', 'ieee', 'signal', 'signal form', 'signal direction', 'signals passed', 'convenient signals', 'modulated signals', 'tech', 'phone', 'mobile phones', 'transmit', 'transmit data', 'information transmitted